In [51]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn.image import load_img, resample_to_img, index_img
from nilearn.masking import intersect_masks, apply_mask, unmask
from nilearn.signal import clean
from nilearn.interfaces.fmriprep import load_confounds
import glob
import re
import matplotlib.pyplot as plt


In [ ]:
# -----------------------------------------------------------------------------
# 1) USER PARAMETERS
# -----------------------------------------------------------------------------

# Where your fmriprep lives
base_dir    = '/Volumes/Passport/fmriprep'
task_label  = 'black'
output_mask = os.path.join(base_dir,
                           f'group_task-{task_label}_intersect_mask.nii.gz')


# Exclusions
exclude_subs     = []
exclude_sub_runs = []

# Which BOLD space / resolution to use
preferred_space = 'MNI152NLin2009cAsym'
preferred_res   = 'native'

target_subject = None               # Keep None unless running on one particular subject


In [53]:
# Expected # subject-run CSVs that should exist after NNLS per-parcel analysis
EXPECTED_COUNT = {
    "slumlordreach":   17,
    "pieman":          75,
    "black":           46,
    "forgot":          46,
    "reachforstars":   17,
    "notthefallintact":54,
}

In [54]:
# -----------------------------------------------------------------------------
# 2) BUILD LIST OF SUBJECTS
# -----------------------------------------------------------------------------
all_subs = sorted(d for d in os.listdir(base_dir)
                  if d.startswith('sub-')
                  and os.path.isdir(os.path.join(base_dir, d)))
if target_subject:
    if target_subject not in all_subs:
        raise ValueError(f"{target_subject} not found under {base_dir}")
    subjects = [target_subject]
else:
    subjects = [s for s in all_subs if s not in exclude_subs]

In [55]:
# -----------------------------------------------------------------------------
# 3) MAKE GROUP INTERSECT MASK
# -----------------------------------------------------------------------------
# where to save the mask
deriv_mask_dir = os.path.join(base_dir, 'derivatives', f'{task_label}_masks')
os.makedirs(deriv_mask_dir, exist_ok=True)
# redefine output_mask to live under derivatives/
output_mask = os.path.join(
    deriv_mask_dir,
    f'group_task-{task_label}_intersect_mask.nii.gz'
)

mask_files = []
space_res  = f"space-{preferred_space}_res-{preferred_res}"
for sub in subjects:
    func_dir = os.path.join(base_dir, sub, 'func')
    # try run-level then single-run masks
    patterns = [
        os.path.join(func_dir,
                     f"{sub}_task-{task_label}_run-1_{space_res}_desc-brain_mask.nii.gz"),
        os.path.join(func_dir,
                     f"{sub}_task-{task_label}_{space_res}_desc-brain_mask.nii.gz"),
    ]
    files = sorted(glob.glob(patterns[0])) or sorted(glob.glob(patterns[1]))
    # make sure files are found
    
    # if not files:
    #     print(f"Skipping: {sub} no mask found")
    
    # get file we want (ie., select run-1, not run-2)
    for fp in files:
        m   = re.search(r"_run-(\d+)_", fp)
        run = m.group(1) if m else None
        if run and (sub,run) in exclude_sub_runs:
            print(f"  • skipping {sub} run-{run}")
            continue
        mask_files.append(fp)

if not mask_files:
    raise RuntimeError("No masks found after exclusions!")

print(f"Found {len(mask_files)} masks, intersecting…")
intersect = intersect_masks(mask_files,
                           threshold=1.0,    
                           connected=False)
intersect.to_filename(output_mask)

mask_img = load_img(output_mask)

print("Saved group intersect mask to:", output_mask)

Found 46 masks, intersecting…
Saved group intersect mask to: /Volumes/Passport/fmriprep/derivatives/black_masks/group_task-black_intersect_mask.nii.gz


In [49]:
# assert we have right number of files per stimulus 
n_expected = EXPECTED_COUNT.get(task_label)

if n_expected is not None:                       # we listed this stim above
    assert len(mask_files) == n_expected, (
        f"[{task_label}] expected {n_expected} files "
        f"but found {len(mask_files)}"
    )
else:                                            # stim not in table → just sanity-check
    if len(mask_files) == 0:
        raise ValueError(f"No CSVs found for {task_label}")

In [50]:
len(mask_files)

46